In [61]:

import torch
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import pandas as pd
import torch
from torch.utils.data import Dataset
import numpy as np
from torch.utils.data import DataLoader
import os

'''
human keypoints: 17*2
court keypoints: 6*2
net keypoints:   4*2
ball keypoints:  1*2
total keypoints: 28*2

'''




class HitDataset(Dataset):
    def __init__(self, dataset_folder,num_consecutive_frames,normalization=True):
        self.dataset=[]
        self.positive=0
        self.negative=0
        # 遍历文件夹及其子文件夹，找到所有的CSV文件路径
        cnt=0
        for root, dirs, files in os.walk(dataset_folder):
            for file in files:
                if file.endswith(".csv"):
                    # 定义处理函数
                    data_path=os.path.join(root, file)
                    print(data_path)
                    
                    df= pd.read_csv(data_path, converters={"ball": eval,"top":eval,"bottom":eval,"court":eval,"net":eval})
                    
                    rows = len(df)
                    remainder = rows % 12
                    if remainder > 0:
                        num_to_pad = 12 - remainder
                    else:
                        num_to_pad = 0

                    if num_to_pad > 0:
                        last_row = df.iloc[-1]
                        padding_data = np.tile(last_row.values, (num_to_pad, 1))
                        padded_df = pd.DataFrame(padding_data, columns=df.columns)
                        df = pd.concat([df, padded_df], axis=0)
                        df = df.reset_index(drop=True)

                    small_dataset =df
                    
                    for i in range(len(small_dataset)):
                        if i%num_consecutive_frames!=0:
                            continue
                        if i>=len(small_dataset)-num_consecutive_frames:
                            break
                        oridata=small_dataset.loc[i:i+num_consecutive_frames-1,:].copy()
                        oridata=oridata.reset_index(drop=True)
                        data=[]
                        
                        target1=None
                        target2=[]
                        
                        for index,row in oridata.iterrows():
                            pos=np.array(row['pos'])
                            if str(pos)=='top': 
                                if target1 is None:
                                    target1=[0.0,1.0,0.0]
                                    self.positive+=1
                                target2.append(1)
                            elif str(pos)=='bottom':
                                if target1 is None:
                                    target1=[0.0,0.0,1.0]
                                    self.positive+=1
                                target2.append(1)
                            else:
                                target2.append(0)
                                    
                        
                            top=np.array(row['top']).reshape(-1,2)
                            bottom=np.array(row['bottom']).reshape(-1,2)
                            court=np.array(row['court']).reshape(-1,2)
                            # net=np.array(row['net']).reshape(-1,2)
                            ball=np.array(row['ball']).reshape(-1,2)
                            
                            frame_data = np.concatenate((top, bottom, court, ball), axis=0)
                            
                            if normalization:
                                frame_data[:,0]/=1920
                                frame_data[:,1]/=1080
                            data.append(frame_data.reshape(1,-1))
                        data=np.array(data)
                        if target1 is None:
                            if self.negative>self.positive:
                                continue
                            target1=[1.0,0.0,0.0] 
                            self.negative+=1                        
                        self.dataset.append((data.reshape(-1),(target1,target2)))
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        # 假设每个样本是一个元组 (input, target)
        sample = self.dataset[index]
        input_data = sample[0]
        target1 = sample[1][0]
        target2 = sample[1][1]
        
        # 转换为Tensor对象
        input_tensor = torch.tensor(input_data)
        target1_tensor = torch.tensor(target1)
        target2_tensor = torch.tensor(target2)

        return input_tensor, (target1_tensor,target2_tensor)
    

num_consecutive_frames=12
batch_size=30
shuffle=True
normalization=True

TrainDataset=HitDataset("ShuttleSet\ShuttleSet22\dataset/train",num_consecutive_frames,normalization)
TestDataset=HitDataset("ShuttleSet\ShuttleSet22\dataset/test",num_consecutive_frames,normalization)



ShuttleSet\ShuttleSet22\dataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals\rally_1-1.csv
ShuttleSet\ShuttleSet22\dataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals\rally_1-11.csv
ShuttleSet\ShuttleSet22\dataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals\rally_1-12.csv
ShuttleSet\ShuttleSet22\dataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals\rally_1-14.csv
ShuttleSet\ShuttleSet22\dataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals\rally_1-16.csv
ShuttleSet\ShuttleSet22\dataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals\rally_1-17.csv
ShuttleSet\ShuttleSet22\dataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals\rally_1-18.csv
ShuttleSet\ShuttleSet22\dataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals\rally_1-19.csv
ShuttleSet\ShuttleSet22\d

In [76]:
class HitModel(nn.Module):
    def __init__(self, feature_dim, num_consecutive_frames):
        super(HitModel, self).__init__()
        self.num_consecutive_frames = num_consecutive_frames
        self.feature_dim = feature_dim
        
        self.gru1 = nn.GRU(feature_dim // num_consecutive_frames, 64, bidirectional=True, batch_first=True)
        self.gru2 = nn.GRU(128, 64, bidirectional=True, batch_first=True)
        self.global_maxpool = nn.MaxPool1d(num_consecutive_frames)
        self.pos = nn.Linear(128, 3)
        self.frame=nn.Linear(128,num_consecutive_frames)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        batch_size = x.shape[0]
        x=x.float()
        x = x.view(batch_size, self.num_consecutive_frames, self.feature_dim // self.num_consecutive_frames)
        x, _ = self.gru1(x)
        x, _ = self.gru2(x)
        x = x.transpose(1, 2)
        x = self.global_maxpool(x).squeeze()
        pos= self.pos(x)
        frame=self.frame(x)
        pos = self.softmax(pos)
        frame=self.softmax(frame)
        return pos,frame

print(TrainDataset.positive,TrainDataset.negative)
print(TestDataset.positive,TestDataset.negative)

feature_dim=82*num_consecutive_frames
train_data_loader = DataLoader(TrainDataset, batch_size=batch_size, shuffle=shuffle)

test_data_loader = DataLoader(TestDataset, batch_size=batch_size, shuffle=shuffle)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=HitModel(feature_dim,num_consecutive_frames)
criterion = nn.CrossEntropyLoss()
model.to(device)
criterion.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 100



train_loss_list = []
test_loss_list=[]
for epoch in range(num_epochs):
    train_loss_sum=0
    model.train()
    for batch_data in train_data_loader:
        inputs, labels = batch_data
        pos=labels[0]
        frame=labels[1]
        inputs = inputs.to(device)  # 将输入数据移动到设备上
        frame = frame.to(device).float()  # 将标签移动到设备上
        pos = pos.to(device).float()  # 将标签移动到设备上
        
        pos_pred,frame_pred = model(inputs)
        pos_pred=pos_pred.reshape(-1,3)
        frame_pred=frame_pred.reshape(-1,num_consecutive_frames)
        
        train_loss = criterion(frame_pred,frame)+criterion(pos_pred,pos)
        train_loss_sum+=train_loss.detach() 
        # 反向传播和优化
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        # 执行你的训练或测试操作
    train_loss_list.append(train_loss_sum)

        
    # 打印训练信息
    if (epoch + 1) % 1== 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs,
                                                    train_loss.item()))
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_data_loader:
            frame=labels[0]
            pos=labels[1]
            inputs = inputs.to(device)  # 将输入数据移动到设备上
            frame = frame.to(device).float()  # 将标签移动到设备上
            pos = pos.to(device).float()  # 将标签移动到设备上

            # 前向传播
            pos_pred,frame_pred = model(inputs)
            pos_pred=pos_pred.reshape(-1,3)
            frame_pred=frame_pred.reshape(-1,num_consecutive_frames)
            
            y_true=torch.argmax(pos,axis=1)
            y_pred=torch.argmax(pos_pred,axis=1)
            total+=len(y_true)
            correct+=(y_true==y_pred).sum().item()
        print(total)
        if total==0:
            print(f'Accuracy on test set: {0}')
            continue
        
            
        
        print(f'Accuracy on test set: {correct/total}')

8568 1131
650 53
Epoch [1/100], Loss: 4.1440
703
Accuracy on test set: 0.05405405405405406
Epoch [2/100], Loss: 4.5054
703
Accuracy on test set: 0.03982930298719772
Epoch [3/100], Loss: 4.8443
703
Accuracy on test set: 0.04267425320056899
Epoch [4/100], Loss: 3.7839
703
Accuracy on test set: 0.08534850640113797
Epoch [5/100], Loss: 3.4700
703
Accuracy on test set: 0.05832147937411095
Epoch [6/100], Loss: 4.4512
703
Accuracy on test set: 0.07539118065433854
Epoch [7/100], Loss: 4.9140
703
Accuracy on test set: 0.07112375533428165


KeyboardInterrupt: 

In [56]:
# 保存整个模型
torch.save(model, 'hitdetect.pth')
# print(f"best_accuracy={best_accuracy}")
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_data_loader:
        inputs = inputs.to(device)  # 将输入数据移动到设备上
        labels = labels.to(device)  # 将标签移动到设备上
        
        # 前向传播
        outputs = model(inputs)
        
        y_true=torch.argmax(labels,axis=1)
        y_pred=torch.argmax(outputs,axis=1)
        print(y_true)
        print(y_pred)
        total+=len(y_true)
        correct+=(y_true==y_pred).sum().item()
        
    
    print(f'Accuracy on test set: {correct/total}')




best_accuracy=0.9075391180654339
tensor([1, 1, 1, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 1], device='cuda:0')
tensor([1, 1, 1, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 1], device='cuda:0')
tensor([1, 2, 1, 1, 2, 2, 1, 2, 2, 2, 1, 0, 2, 2, 1], device='cuda:0')
tensor([1, 2, 1, 1, 2, 2, 1, 2, 2, 2, 1, 0, 2, 2, 1], device='cuda:0')
tensor([1, 2, 1, 0, 2, 1, 1, 2, 0, 1, 1, 1, 1, 2, 0], device='cuda:0')
tensor([1, 2, 1, 0, 1, 0, 2, 2, 2, 1, 1, 1, 1, 2, 0], device='cuda:0')
tensor([2, 1, 1, 2, 1, 2, 1, 0, 1, 0, 0, 2, 1, 1, 1], device='cuda:0')
tensor([2, 1, 1, 2, 1, 2, 1, 1, 2, 0, 1, 2, 2, 1, 1], device='cuda:0')
tensor([1, 1, 1, 2, 2, 0, 1, 2, 1, 1, 2, 1, 2, 2, 1], device='cuda:0')
tensor([1, 1, 1, 2, 2, 0, 1, 2, 1, 1, 2, 1, 2, 2, 1], device='cuda:0')
tensor([2, 1, 2, 2, 1, 0, 1, 1, 1, 1, 0, 1, 1, 2, 1], device='cuda:0')
tensor([2, 1, 2, 2, 1, 0, 1, 1, 1, 1, 2, 1, 1, 2, 1], device='cuda:0')
tensor([0, 2, 2, 1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1, 0], device='cuda:0')
tensor([0, 2, 2, 1, 2, 2, 2, 1, 1, 1, 2, 1, 

In [57]:
final_list=[]
true_list=[]
data_path="ShuttleSet\ShuttleSet22\dataset/valid\AN_Se_Young_Gregoria_Mariska_TUNJUNG_Malaysia_Masters_2022_SemiFinals/rally_1-1.csv"
data_path="ShuttleSet\ShuttleSet22\dataset/valid\AN_Se_Young_Gregoria_Mariska_TUNJUNG_Malaysia_Masters_2022_SemiFinals/rally_3-27.csv"
df= pd.read_csv(data_path, converters={"ball": eval,"top":eval,"bottom":eval,"court":eval,"net":eval})

rows = len(df)
remainder = rows % 12
if remainder > 0:
    num_to_pad = 12 - remainder
else:
    num_to_pad = 0

if num_to_pad > 0:
    last_row = df.iloc[-1]
    padding_data = np.tile(last_row.values, (num_to_pad, 1))
    padded_df = pd.DataFrame(padding_data, columns=df.columns)
    df = pd.concat([df, padded_df], axis=0)
    df = df.reset_index(drop=True)

small_dataset =df
probs_list=[]
for i in range(len(small_dataset)):
    
    if i>=len(small_dataset)-num_consecutive_frames:
        break
    
    
    oridata=small_dataset.loc[i:i+num_consecutive_frames-1,:].copy()
    oridata=oridata.reset_index(drop=True)
    data=[]
    # print(oridata)
    if str(oridata.loc[0,'pos'])=='nan':
            true_list.append(0)
    elif str(oridata.loc[0,'pos'])=='top':
            true_list.append(1)
    elif str(oridata.loc[0,'pos'])=='bottom':
            true_list.append(2)
        
    for index,row in oridata.iterrows():
        
        top=np.array(row['top']).reshape(-1,2)
        bottom=np.array(row['bottom']).reshape(-1,2)
        court=np.array(row['court']).reshape(-1,2)
        # net=np.array(row['net']).reshape(-1,2)
        ball=np.array(row['ball']).reshape(-1,2)
        
        frame_data = np.concatenate((top, bottom, court, ball), axis=0)
        
        if normalization:        
            # 将 x 坐标从 0 到 1920 映射到 1 到 2
            frame_data[:,0] /=1920

            # 将 y 坐标从 0 到 1080 映射到 1 到 2
            frame_data[:,1] /=1080

        data.append(frame_data.reshape(1,-1))
    data=np.array(data).reshape(1,-1)
    # print(data.shape)
    # if i%num_consecutive_frames!=0:
    #     final_list.append(0)
    #     continue     
    outputs=model(torch.FloatTensor(data).to(device))
    
    # print(outputs)
    pred=torch.argmax(outputs).item()
    probs=outputs[pred].item()
    final_list.append(pred)

print(true_list)
print(final_list)    


[0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1,